# Attempting a Differential Attack on PRESENT-80 with MILP using Convex Hull of S-Box

### SETUP

Importing libraries, setting up functions, preparing the encryption standard as guided with simple_spn.ipynb

In [1]:
import secrets  # for randbits
from time import time
import numpy as np
import matplotlib.pyplot as plt
from inspect import signature
from itertools import product
from functools import reduce
from sage.crypto.block_cipher.present import PRESENT, PRESENT_KS

In [2]:
tobits= lambda x, nbits: [*map(int, format(x, "0%db"%nbits)[::-1])]     # little endian

frombits= lambda y: sum(bit*(1<<i) for i,bit in enumerate(y))

def perm(pbox, cipbits):
    assert len(cipbits) == len(pbox)
    return [cipbits[pbox[i]] for i in range(len(cipbits))]


def get_ddt(sbox):
    l= len(sbox)
    ddt= np.zeros((l,l), dtype=int)
    for i,x in enumerate(sbox):
        for j,y in enumerate(sbox):
            ddt[i^^j][x^^y]+= 1
    return ddt


def get_sbox_hrep(sbox, threshold:int=0):
    assert threshold >= 0, "Threshold must be a positive integer"
    assert threshold <= len(sbox), "Threshold= `threshold` is larger than SBOX!"
    ddt = get_ddt(sbox)
    l = len(sbox)
    lb = int(l).bit_length()-1
    assert l == 1<<lb, "Size of SBOX isn't a power of two!"   
    # each successive bit carries an extra weight of *2 => l= max(weight) of bits

    space = []
    for x in range(l):
        for y in range(l):
            if ddt[x,y] <= threshold: continue
            xb = tobits(x, lb)
            yb = tobits(y, lb)
            space.append((*xb, *yb))
    
    p = Polyhedron(vertices=space)
    return p.Hrepresentation()


def hrep_to_ineq(hrep, lin_var):
    assert len(lin_var) == len(hrep[0]) - 1, \
        "Number of arguments does not match dimensionality of `hrep`"
    # hrep includes constant b in iterable as element 0
    return [
        sum(var*coeff for coeff,var in zip([*ieq][1:], lin_var)) + ieq.b() >= 0 
        for ieq in hrep
    ]
    # 1.    The function pushes '*' multiply operator. MIP linear function (lf) 
    #       attaches real constants to variables for storage as lf object. 
    # 2.    '>=', __geq__ has been overwritten to read 2 lf and output some 
    #       linear constraint (lc), thus NOT a boolean
    # 3.    Returns a list of constraints


def toblks(arr, blocklen:int):
    assert len(arr)%blocklen ==0, "`blocklen` isn't a divisor of `len(arr)`"
    return [arr[i : i + blocklen] for i in range(0, len(arr), blocklen)]


def get_boolexpr_hrep(boolfunc):
    """
    Get H-Representation of points representing
    (a0,a1,...,an, boolfunc(a0,a1,...,an))
    
    boolfunc is a boolean function of n-args with a boolean output
    """
    nargs = len(signature(boolfunc).parameters)
    space = []
    for nb in product([0,1], repeat=nargs): # all possible binary vectors in n-d
        space.append((*nb, boolfunc(*nb)))
        
    p = Polyhedron(vertices=space)  # entire space based on the defined bool func
    return p.Hrepresentation()
    

prod= lambda arr: reduce(lambda x,y: x*y, arr)

In [3]:
class VarGen:
    
    """
    Wrapper class over `solver.new_variable`
    to provide the `gen` method
    """
    
    def __init__(self, solver:MixedIntegerLinearProgram):
        self.vargen = solver.new_variable(integer=True)
        
    def __getitem__(self, idx):
        """Get an existing variable at index `idx`"""
        assert idx < len(self.vargen.keys())
        return self.vargen[idx]
    
    def gen(self):
        """Generates a new variable"""
        return self.vargen[len(self.vargen.keys())]


In [ ]:
RDS= 4

SBOX= [0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD, 0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2]  # index= original 4-bit word
INV_SBOX= [SBOX.index(i) for i in range(16)]

PBOX= [ (16*i) %63 for i in range(63)] + [63]
INV_PBOX= [PBOX.index(i) for i in range(16)]